In [42]:
%pip install wand
!apt-get install libmagickwand-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [82]:
import numpy as np
import torch
import matplotlib.pyplot as plt
np.random.seed(2)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Generate training dataset
T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')

In [83]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
  """
  Define LTSM model to predict. There are 2 LSTM layers and 1 linear layer
  input -> LSTM -> LSTM -> Linear -> output
  Output of LSTM haing size 51 to capture the frequency of sine function,
  then these features are pushed into linear layer to predict 1 next value
  """
  def __init__(self):
    super(Sequence, self).__init__()
    self.lstm1 = nn.LSTMCell(1, 51)
    self.lstm2 = nn.LSTMCell(51, 51)
    self.linear = nn.Linear(51, 1)
  
  def forward(self, input, future = 0):
      outputs = []
      h_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device) # shape 999 x 51
      c_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
      h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
      c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)

      for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
          h_t, c_t = self.lstm1(input_t, (h_t, c_t))
          h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
          output = self.linear(h_t2) # Matrix shape 97 x 1
          # after loop: vector outputs shape: (999,), each element is output with shape 97 x 1
          outputs += [output] 

      for i in range(future):# if we should predict the future
          h_t, c_t = self.lstm1(output, (h_t, c_t))
          h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
          output = self.linear(h_t2)
          outputs += [output]

      outputs = torch.stack(outputs, 1) # shape 97 x 999 x 1
      outputs = outputs.squeeze(2) # shape 97 x 999
      return outputs

In [84]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
input = torch.from_numpy(data[3:, :-1]).to(device)
target = torch.from_numpy(data[3:, 1:]).to(device)
test_input = torch.from_numpy(data[:3, :-1]).to(device)
test_target = torch.from_numpy(data[:3, 1:]).to(device)

In [85]:
input.shape, target.shape

(torch.Size([97, 999]), torch.Size([97, 999]))

In [86]:
# build the model
seq = Sequence().to(device)
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
for i in range(9):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    break
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.cpu().detach().numpy()

STEP:  0
loss: 0.5023738149789433
loss: 0.49856639650504164
loss: 0.47901196321589745
loss: 0.44633490457512764
loss: 0.35406310449995715
loss: 0.20507016729177277


KeyboardInterrupt: ignored

In [ ]:
# draw the result
plt.figure(figsize=(30,10))
plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
def draw(yi, color):
      plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
      plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
draw(y[0], 'r')
draw(y[1], 'g')
draw(y[2], 'b')
plt.savefig('predict%d.pdf'%i)
plt.close()